In [13]:
# I tried  this package https://pypi.org/project/quora-scraper/  
# and after going through installation steps 
# I faced some problems so I copied the code from the package
# and made some modifications

In [3]:
# Mount the Google drive
from google.colab import drive # this will be our driver
drive.mount('/gdrive')
root = '/gdrive/My Drive/'     # if you want to operate on your Google Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [1]:
# install selenium package
!pip install selenium

In [3]:
# Download Chromedriver because this package require it
!wget https://chromedriver.storage.googleapis.com/85.0.4183.38/chromedriver_linux64.zip  && unzip chromedriver_linux64

--2020-08-07 23:12:04--  https://chromedriver.storage.googleapis.com/85.0.4183.38/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 173.194.217.128, 2607:f8b0:400c:c13::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5358213 (5.1M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   5.11M  --.-KB/s    in 0.03s   

2020-08-07 23:12:04 (158 MB/s) - ‘chromedriver_linux64.zip’ saved [5358213/5358213]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            


In [4]:
# We set a variable with the *Colab* default path and chrome driver path, to easily use it later
colab_path = '/content/'
chromedriver_path = '/content/chromedriver'

In [6]:
# add chromedriver to sys path
import sys
sys.path.insert(0,chromedriver_path)

In [7]:
# show sys path
print(sys.path)

['/content/chromedriver', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [8]:
# try chrome driver and adjusting some options such that it does not crash in google colab
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [9]:
# test webdriver
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get("https://quantamixsolutions.com/")
driver.get_screenshot_as_file("screenshot.png")
print("We are in ",driver.title)
driver.quit()


We are in  Become Your Market's Leading Authority on Page #1 of Google | Quantamix Solutions


In [18]:
# modified version of https://github.com/banyous/quora-scraper/blob/master/quora_scraper/scraper.py
import os
import re
import subprocess
import sys
import time
import ast
import csv
import json
import pathlib
from pathlib import Path
import random
import userpaths
import dateparser
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
# -------------------------------------------------------------
# -------------------------------------------------------------

def connectchrome():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("--start-maximized")
  driver = webdriver.Chrome('chromedriver',options=chrome_options)
  return driver


# -------------------------------------------------------------
# -------------------------------------------------------------
# remove 'k'(kilo) and 'm'(million) from Quora numbers
def convertnumber(number):
	if 'k' in number:
		n=float(number.lower().replace('k', '').replace(' ', ''))*1000
	elif 'm' in number:
		n=float(number.lower().replace('m', '').replace(' ', ''))*1000000
	else:
		n=number
	return int(n)
# -------------------------------------------------------------
# -------------------------------------------------------------
# convert Quora dates (such as 2 months ago) to DD-MM-YYYY format
def convertDateFormat(dateText):
	try:
		if ("Updated" in dateText):
			date = dateText[8:]
		else:
			date = dateText[9:]
		date = dateparser.parse(dateText).strftime("%Y-%m-%d")
	except:  # when updated or answered in the same week (ex: Updated Sat)
		date = dateparser.parse("7 days ago").strftime("%Y-%m-%d")
	return date

# -------------------------------------------------------------
# -------------------------------------------------------------
def scrollup_alittle(self,nbtimes):
	
	for iii in range(0,nbtimes):
		self.execute_script("window.scrollBy(0,-200)")
		time.sleep(1)
# -------------------------------------------------------------
# -------------------------------------------------------------
# method for loading  quora dynamic content
def scrolldown(self,type_of_page='users'):
	last_height = self.page_source
	loop_scroll=True
	attempt = 0
	# we generate a random waiting time between 2 and 4
	waiting_scroll_time=round(random.uniform(2, 4),1)
	print('scrolling down to get all answers...')
	max_waiting_time=round(random.uniform(5, 7),1)
	# we increase waiting time when we look for questions urls	
	if type_of_page=='questions' : max_waiting_time= round(random.uniform(20, 30),1)
	# scroll down loop until page not changing
	while loop_scroll:
		self.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(2)
		if type_of_page=='answers':
			scrollup_alittle(self,2)
		new_height=self.page_source
		if new_height == last_height:
			# in case of not change, we increase the waiting time
			waiting_scroll_time= max_waiting_time
			attempt += 1
			if attempt==3:# in the third attempt we end the scrolling
				loop_scroll=False
			#print('attempt',attempt)
		else:
			attempt=0
			waiting_scroll_time=round(random.uniform(2, 4),1)
		last_height=new_height

# -------------------------------------------------------------
# -------------------------------------------------------------	
# questions urls crawler 
def questions(topics_list,save_path):
	browser=connectchrome()
	topic_index=-1
	loop_limit=len(topics_list)
	print('Starting the questions crawling')
	while True:
		print('--------------------------------------------------')
		topic_index += 1
		if topic_index>=loop_limit:
			print('Crawling completed, questions have been saved to  :  ', save_path)
			browser.quit()
			break
		topic_term = topics_list[topic_index].strip()
		# we remove hashtags (optional)
		topic_term.replace("#",'')
		# Looking if the topic has an existing Quora url
		print('#########################################################')
		print('Looking for topic number : ',topic_index,' | ', topic_term)
		try:
			url = "https://www.quora.com/topic/" + topic_term.strip() + "/all_questions"
			browser.get(url)
			time.sleep(2)
		except Exception as e0:
			print('topic does not exist in Quora')
			# print('exception e0')
			# print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e0).__name__, e0)
			continue

		# get browser source
		html_source = browser.page_source
		question_count_soup = BeautifulSoup(html_source, 'html.parser')

		#  get total number of questions
		question_count_str = question_count_soup.find('a', attrs={'class': 'TopicQuestionsStatsRow'})
		if str(question_count_str) =='None':
			print('topic does not have questions...')
			continue
		question_count = convertnumber(question_count_str.contents[0].text)
		question_count_str = question_count_soup.find('a', attrs={'class': 'TopicQuestionsStatsRow'})
		if question_count ==0:
			print('topic does not have questions...')
			continue
		print('number of questions for this topic : '+ str(question_count))

		# Get scroll height
		last_height = browser.execute_script("return document.body.scrollHeight")

		# infinite while loop, break it when you reach the end of the page or not able to scroll further.
		# Note that Quora
		# if there is more than 10 questions, we need to scroll down the profile to load remaining questions
		if int(question_count)>10: 
			scrolldown(browser,'questions')

		# next we harvest all questions URLs that exists in the Quora topic's page
		# get html page source
		html_source = browser.page_source
		soup = BeautifulSoup(html_source, 'html.parser')

		# question_link is the class for questions
		question_link = soup.find_all('a', attrs={'class': 'question_link'}, href=True)

		# add questions to a set for uniqueness
		question_set = set()
		for ques in question_link:
			question_set.add(ques)

		# write content of set to Qyestions_URLs/ folder
		save_file= Path(save_path) /  str(topic_term.strip('\n') + '_question_urls.txt')
		file_question_urls = open(save_file, mode='w', encoding='utf-8')
		for ques in question_set:
			link_url = "http://www.quora.com" + ques.attrs['href']
			file_question_urls.write(link_url+'\n')
		file_question_urls.close()
		
		# sleep every while in order to not get banned
		if topic_index % 5 == 4:
			sleep_time = (round(random.uniform(5, 10), 1))
			time.sleep(sleep_time)

	browser.quit()
 

# -------------------------------------------------------------
# -------------------------------------------------------------
# answers cralwer
def answers(urls_list,save_path):
	browser= connectchrome()
	url_index = -1
	loop_limit= len(urls_list)
	# output file containing all answers
	file_answers = open(Path(save_path) / "answers.txt", mode='a') 
	print('Starting the answers crawling...')
	while True:
		url_index += 1
		print('--------------------------------------------------')
		if url_index >= loop_limit:
			print('Crawling completed, answers have been saved to  :  ', save_path)
			browser.quit()
			file_answers.close()
			break
		current_line = urls_list[url_index]
		print('processing question number  : '+ str(url_index+1))
		print(current_line)
		if '/unanswered/' in str(current_line):
			print('answer is unanswered')
			continue
		question_id = current_line
		# opening Question page
		try:
			browser.get(current_line)
			time.sleep(2)
		except Exception as OpenEx:
			print('cant open the following question link : ',current_line)
			print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(OpenEx).__name__, OpenEx)
			print(str(OpenEx))
			continue
		try:
			nb_answers_text = WebDriverWait(browser, 10).until(
			EC.visibility_of_element_located((By.XPATH, "//div[text()[contains(.,'Answer')]]"))).text
			nb_answers=[int(s.strip('+')) for s in nb_answers_text.split() if s.strip('+').isdigit()][0]
			print('Question have :', nb_answers_text)
		except Exception as Openans: 
			print('cant get answers')
			print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(Openans).__name__, Openans)
			print(str(Openans))
			continue
		#nb_answers_text = browser.find_element_by_xpath("//div[@class='QuestionPageAnswerHeader']//div[@class='answer_count']").text
		
		if nb_answers>7:
			scrolldown(browser,'answers')
		continue_reading_buttons = browser.find_elements_by_xpath("//a[@role='button']")
		time.sleep(2)
		for button in continue_reading_buttons:
			try:
				ActionChains(browser).click(button).perform()
				time.sleep(1)
			except:
				print('cant click more')
				continue
		time.sleep(2)
		html_source = browser.page_source
		soup = BeautifulSoup(html_source,"html.parser")
		# get the question-id
		question_id = current_line.rsplit('/', 1)[-1]
		# find title 
		title= current_line.replace("https://www.quora.com/","")
		# find question's topics
		questions_topics= soup.findAll("div", {"class": "q-box qu-mr--tiny qu-mb--tiny"})
		questions_topics_text=[]
		for topic in questions_topics : questions_topics_text.append(topic.text.rstrip())
		# number of answers
		# not all answers are saved!
		# answers that collapsed, and those written by annonymous users are not saved
		try:
			split_html = html_source.split('class="q-box qu-pt--medium qu-pb--medium"')
		except Exception as notexist :#mostly because question is deleted by quora
			print('question no long exists')
			print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(notexist).__name__, notexist)
			print(str(notexist))
			continue	
		# The underneath loop will generate len(split_html)/2 exceptions, cause answers in split_html
		# are eitheir in Odd or Pair positions, so ignore printed exceptions.
		#print('len split : ',len(split_html))
		for i in range(1, len(split_html)):
			try:
				part = split_html[i]
				part_soup = BeautifulSoup(part,"html.parser" )
				#print('===============================================================')
				#find users names of answers authors
				try:				
					authors=part_soup.find("a", href=lambda href: href and "/profile/" in href)
					user_id = authors['href'].rsplit('/', 1)[-1]
					#print(user_id)
				except Exception as notexist2 :#mostly because question is deleted by quora
					print('author extract pb')
					print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(notexist2).__name__, notexist2)
					print(str(notexist2))
					continue
					
				# find answer dates
				
				answer_date= part_soup.find("a", string=lambda string: string and ("Answered" in string or "Updated" in string))#("a", {"class": "answer_permalink"})
				try:
					date=answer_date.text
					if ("Updated" in date):
					   date= date[8:]
					else:
					   date= date[9:]
					date=dateparser.parse(date).strftime("%Y-%m-%d")
				except: # when updated or answered in the same week (ex: Updated Sat)
					date=dateparser.parse("7 days ago").strftime("%Y-%m-%d")
				#print(date)
				# find answers text
				answer_text = part_soup.find("div", {"class": "q-relative spacing_log_answer_content"})
				#print(" answer_text", answer_text.text)
				answer_text = answer_text.text
				#write answer elements to file
				s=  str(question_id.rstrip()) +'\t' + str(date) + "\t"+ user_id + "\t"+ str(questions_topics_text) + "\t" +	str(answer_text.rstrip())  + "\n"
				#print("wrting down the answer...")
				file_answers.write(s)
				print('writing down answers...')
			except Exception as e1: # Most times because user is anonymous ,  continue without saving anything
				print('---------------There is an Exception-----------')
				print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e1).__name__, e1)
				print(str(e1))
				o=1
		
		# we sleep every while in order to avoid IP ban
		if url_index%3==2:
			sleep_time=(round(random.uniform(5, 10),1))
			time.sleep(sleep_time)
	browser.quit()


In [11]:
# scraping quora answers
# Input is a list of Questions URL and the path to save the output
# Output is a file of scraped answers (answers.txt). An answer consists of :
# Quest-ID | AnswerDate | AnswerAuthor-ID | Quest-tags | Answer-Text
questions = ['https://www.quora.com/What-is-artificial-intelligence-15']
save_path = colab_path;
answers(questions,save_path)

Starting the answers crawling...
--------------------------------------------------
processing question number  : 1
https://www.quora.com/What-is-artificial-intelligence-15
Question have : 100+ Answers
scrolling down to get all answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answers...
writing down answ

In [19]:
# scraping questions related to certain topics
# input is a list of topic keywords.
# output is a file containing the topic's question links.
keywords = ['semantic seo','seo']
questions(keywords,save_path)

Starting the questions crawling
--------------------------------------------------
#########################################################
Looking for topic number :  0  |  semantic seo
topic does not have questions...
--------------------------------------------------
#########################################################
Looking for topic number :  1  |  seo
number of questions for this topic : 168000
scrolling down to get all answers...
--------------------------------------------------
Crawling completed, questions have been saved to  :   /content/
